In [0]:


#so we created a ingestion config file for all the sources we have if a source name is change we dont have to change name manually eveywhere in our code and it help to orgnize eveything.

import json
from pyspark.sql import SparkSession

# -------------------------------
#  Load ingestion config
# -------------------------------
config_file_path = "/Workspace/Users/forazure1995@gmail.com/ingestion_config.json"

with open(config_file_path, "r") as f:
    INGESTION_CONFIG = json.load(f)

# -------------------------------
#  Base path for Bronze layer
# -------------------------------
BRONZE_BASE_PATH = "abfss://bronze@fordatabricks111.dfs.core.windows.net/"

# -------------------------------
# Helper function to read any table
# -------------------------------
def read_table(table_name):
    """
    Reads a CSV from the config by logical table name
    Returns a Spark DataFrame
    """
    # Find the path from config
    path = next(cfg["path"] for cfg in INGESTION_CONFIG if cfg["table"] == table_name)
    return spark.read.csv(path, header=True, inferSchema=True)







In [0]:
#now we can just use readtable function and ca read anytable easily
df_prd = read_table("sales_details")
display(df_prd)


In [0]:
%sql
-- ========================================
-- CREATE BRONZE SCHEMA
-- ========================================
CREATE SCHEMA IF NOT EXISTS databrickswp.bronze
MANAGED LOCATION 'abfss://bronze@fordatabricks111.dfs.core.windows.net/'
COMMENT 'Bronze layer - Raw ingested data';

-- ========================================
-- CREATE SILVER SCHEMA
-- ========================================
CREATE SCHEMA IF NOT EXISTS databrickswp.silver
MANAGED LOCATION 'abfss://silver@fordatabricks111.dfs.core.windows.net/'
COMMENT 'Silver layer - Cleaned and validated data';

-- ========================================
-- CREATE GOLD SCHEMA
-- ========================================
CREATE SCHEMA IF NOT EXISTS databrickswp.gold
MANAGED LOCATION 'abfss://gold@fordatabricks111.dfs.core.windows.net/'
COMMENT 'Gold layer - Business-level aggregates and analytics';

-- ========================================
-- VERIFY SCHEMAS CREATED
-- ========================================
SHOW SCHEMAS IN databrickswp;

-- Describe each schema to verify
DESCRIBE SCHEMA EXTENDED databrickswp.bronze;
DESCRIBE SCHEMA EXTENDED databrickswp.silver;
DESCRIBE SCHEMA EXTENDED databrickswp.gold;

write all csv files into bronze layer in parquet format

In [0]:
# Loop through ingestion config instead of hard-coded CSVs
for cfg in INGESTION_CONFIG:
    table_name = cfg["table"]      # logical table name
    source_path = cfg["path"]      # CSV path from config

    # Read CSV
    df = spark.read.csv(source_path, header=True, inferSchema=True)

    # Save to Bronze folder with same name as table
    target_path = f"{bronze_path}{table_name}/"  # use your existing bronze_path variable
    df.write.mode("overwrite").parquet(target_path)

    print(f"{table_name} saved to {target_path} as Parquet")


In [0]:
dfff=read_table("sales_details")
display(dfff)